In [1]:
#MOPSO
import numpy as np
global GLOBAL_BEST

In [2]:
def crowdingDistSorting(children, population):
    """
    Sorts the chldren belonging to population acc. to the crowding distance sorting
    NOTE - Works only for 2 obective functions
    Args:
        Children (2D array of size (?,nvar)) - the subset of population belonging to a particular pareto 
                                                which needs to be sorted
        Population (2D array of size (npop, nvar)) - the current population
    Returns : Sorted Children
    """
    
    
    costValArray = convert2objective(population)

    Smax, Tmax = np.max(costValArray, axis = 0)  #surfaceArea, TotalArea
    Smin, Tmin = np.min(costValArray, axis = 0)

    tmpCrowd = []
    
    tmp = np.copy(children)
    #assign inf distance to the extreme points corresponding to all objective functions
    tmp = sorted(children, key=lambda x:lateralSA(population[x][0], population[x][1]))
    tmpCrowd.append((tmp[0] ,float('inf')))
    tmpCrowd.append((tmp[-1], float('inf')))
    min1 = tmp[0]
    max1 = tmp[-1]
    
    tmp = sorted(children, key=lambda x:totalSA(population[x][0], population[x][1]))
    tmpCrowd.append([tmp[0] ,float('inf')])
    tmpCrowd.append([tmp[-1], float('inf')])
    min2 = tmp[0]
    max2 = tmp[-1]
    
    extremes = {min1, min2, max1, max2}
    for idx, val in enumerate(tmp):
        if val in extremes:
            pass
        else:
            dist = abs(lateralSA(population[tmp[idx+1]][0], population[tmp[idx+1]][1])-
                    lateralSA(population[tmp[idx-1]][0], population[tmp[idx-1]][1]))/(Smax-Smin)+abs(
                    totalSA(population[tmp[idx+1]][0], population[tmp[idx+1]][1]) - 
                    totalSA(population[tmp[idx-1]][0], population[tmp[idx-1]][1]))/(Tmax-Tmin)

            tmpCrowd.append([val, dist])
    
    tmpCrowd = np.array(tmpCrowd)
    newChildren = np.array(sorted(tmpCrowd, key = lambda x:x[1], reverse=True)).squeeze()
    newChildren = newChildren[:,0]
    idx = np.unique(newChildren, return_index = True)[1]
    newChildren = [newChildren[i] for i in sorted(idx)]
    return np.int32(newChildren)

In [3]:
def atLeast1dominate(front, x):
    """
    Checks wheather atleast one soln in front dominates soln x
    Args:
        front (list) - list of soln(with cost values) belonging to a front
        x (array) - a solution containing the cost values
    Returns:
        True/ False
    """
    if True in [(i<x).all() for i in front]:
        return True
    return False
def findFront(idx, paretoFronts, costs):
    """
    Returns the front to which the solution[idx] belongs, and simultaneously updates the Pareto front
    Args:
        idx (int) - the index of the solution whose front no. is to be found
        paretoFronts (dict) - the current paretoFronts, to be updated too.
        costs (2D array)- the cost values array corresponding the current population sorted to descending
                            order of 1 cost function (here lateralSA)
    Return:
        front (int) - the front no. to which soln[idx] belongs
    Update:
        paretoFronts (dict) - assigns idx to ParetoFronts[front]
    """
    l = len(paretoFronts)
    if l==0:
        paretoFronts[1] = [idx]
        return 1
    for i in paretoFronts:
        front = paretoFronts[i]
        if atLeast1dominate([costs[t] for t in front], costs[idx])==False:
            paretoFronts[i].append(idx)
            return idx
    paretoFronts[l+1] = [idx]
    return l+1
def nonDominatedSorting(population, cost1, cost2, convert2objective):
    """
    Perfroms non-dominated sorting on the population and returns the pareto front, along with the population 
    sorted in escending order of lateralSA 
    NOTE - crowding distance sorting not applied here
    """
    nvar = len(population[0].position)
    npop = len(population)
    
    
    populationSorted = np.array(sorted(population, key = lambda x:cost1(x.position))
    costs = convert2objective(populationSorted)
    paretoFronts = {}
    
    for i in range(npop):
        findFront(i, paretoFronts, costs)
    return paretoFronts, populationSorted

def updateRank(paretoFronts):
    """
    Returns the updated rank of each solution
    Args:
        paretoFronts (dict) - current pareto fronts
    Return:
        rank (dict) - rank of each solution idx
    """
    ranks = {}
    r = 1
    for front in paretoFronts:
        for idx in paretoFronts[front]:
            ranks[idx] = r
            r+=1
    return ranks
def updatePopulation(population, paretoFronts, npop):
    newPop = []
    if len(population)<=npop:
        return population
    count=0
    flag = 0
    for front in paretoFronts:
        if flag==1:
            break
        for x in paretoFronts[front]:
            if count>=npop:
                flag=1
                break
            newPop.append(population[x])
            count+=1
    return np.array(newPop)
def plotParetoFronts(paretoFronts, population, convert2objective):
    """
    Plots the pareto fronts from the given pareto front and population data
    """
    costs = convert2objective(population)
    plt.scatter(costs[:,0], costs[:,1])
    for front in paretoFronts:
        cost = np.array([costs[i] for i in paretoFronts[front]])
        S = cost[:,0]
        T = cost[:,1]
        plt.plot(S,T)
        #plt.show()
    plt.xlabel("Lateral Surface Area")
    plt.ylabel("Total Surface Area")
    plt.title("Pareto Fronts")

In [6]:
class particle:
    def __init__(self):
        self.position = None
        self.velocity = None
        self.cost1 = None
        self.cost2 = None
        self.best = None
        

In [19]:
a = particle()
a.position = 5
a.velocity = 6
a.best = a
b = a
del a
b.best.position

5

In [26]:
def initialisePop(npop, cost1, cost2, nvar, varMax, varMin):
    global  GLOBAL-BEST
    population = []
    for i in range(npop):
        x = particle()
        x.position = np.around(np.random.uniform(low=varMin, high = varMax, size = nvar), 2)
        x.velocity = np.zeros(nvar)
        x.cost1 = cost1(x.position)
        x.cost2 = cost2(x.position)
        x.best = x
        population.append(x)
        del x
    return population

In [28]:
#sanity check
pops = initialisePop(3, lambda x:x*0, lambda x:x*0, 2, 0, 1)
for i in pops:
    print (i.best.position, i.best.cost1)

[0.92 0.99] [0. 0.]
[0.46 0.2 ] [0. 0.]
[0.16 0.49] [0. 0.]


In [30]:
def mopsoOptimizer(cost1, cost2, npop = 50, nvar = 2, varMax=1, varMin=0, niter = 200, c1=2, c2=2, w=1, wdamp=0.99):
    global GLOBAL_BEST
    GLOBAL_BEST = float("inf")
    convert2objective = lambda x:np.array([[cost1(i.position), cost2(i.position)] for i in x])
    
    population = initialisePop(npop, cost1, cost2, nvar, varMax, varMin)
    paretoFronts, population = nonDominatedSorting(population, convert2objective)
    

    

SyntaxError: invalid syntax (<ipython-input-30-62f700176cd8>, line 1)

In [2]:
a = 'abc'
a.index("a")

0

In [3]:
a.index("")

0